In [18]:
import torch
import torch.nn as nn
import torch.utils.data as Data
from torch.autograd import Variable

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
#data processing
df_train_txt = pd.read_csv('training_text', sep='\|\|', header=None, skiprows=1, names=["ID","Text"])
df_train_var = pd.read_csv('training_variants')
df_test_txt = pd.read_csv('test_text', sep='\|\|', header=None, skiprows=1, names=["ID","Text"])
df_test_var = pd.read_csv('test_variants')
df_train = pd.merge(df_train_var, df_train_txt, how='left', on='ID')
df_test = pd.merge(df_test_var, df_test_txt, how='left', on='ID')
col = ['ID', 'Gene', 'Variation', 'Text', 'Class']
df_train = df_train.loc[:, col]
df_train.head()

/home/quoniammm/anaconda3/envs/py3Tfgpu/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/home/quoniammm/anaconda3/envs/py3Tfgpu/lib/python3.6/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


,ID,Gene,Variation,Text,Class
0,0,FAM58A,Truncating Mutations,Cyclin-dependent kinases (CDKs) regulate a var...,1
1,1,CBL,W802*,Abstract Background Non-small cell lung canc...,2
2,2,CBL,Q249E,Abstract Background Non-small cell lung canc...,2
3,3,CBL,N454D,Recent evidence has demonstrated that acquired...,3
4,4,CBL,L399V,Oncogenic mutations in the monomeric Casitas B...,4


In [21]:
title = df_train['Gene'].values + ' ' + df_train['Variation'].values
title = title.reshape(len(title), 1).tolist()

In [22]:
content = df_train['Text'].values
content = content.reshape(len(content), 1).tolist()

In [23]:
cla = df_train['Class'].values
cla = cla.reshape(len(cla), 1).tolist()

In [24]:
class Sentence:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.n_words = 2 # Count SOS and EOS
      
    def index_words(self, sentence):
        for word in sentence.split(' '):
            self.index_word(word)

    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [25]:
tit = Sentence('title')
con = Sentence('content')
for t, c in zip (title, content):
    c = ''.join(c)
#     print(c)
    t = ''.join(t)
#     print(t)
    tit.index_words(t)
    con.index_words(c)
    
print('tit: {}'.format(tit.n_words))
print('con: {}'.format(con.n_words))

tit: 3281
con: 496037


In [26]:
def indexes_from_sentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def variable_from_sentence(lang, sentence):
    indexes = indexes_from_sentence(lang, sentence)
    var = Variable(torch.LongTensor(indexes).view(-1, 1))
#     if USE_CUDA: var = var.cuda()
    return var

def variables_from_pair(t, c):
    title = variable_from_sentence(tit, t)
    content = variable_from_sentence(con, c)
    return (title, content)

In [27]:
# USE_CUDA = True

In [28]:
class CNNText(nn.Module): 
    def __init__(self):
        super(CNNText, self).__init__()
        self.encoder_tit = nn.Embedding(3281, 64)
        self.encoder_con = nn.Embedding(496037, 512)

        self.title_conv = nn.Sequential(
            nn.Conv1d(in_channels = ,
                      out_channels = ,
                      kernel_size = ,
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=)
        )

        self.content_conv = nn.Sequential(
            nn.Conv1d(in_channels = 1,
                      out_channels = 1,
                      kernel_size = (2, 512)),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size = (496034))
        )

        self.fc = nn.Linear(2, 9)

    def forward(self, title, content):
        title = self.encoder_tit(title)
        content = self.encoder_con(content)
        title_out = self.title_conv(title)
        content_out = self.content_conv(content)
        conv_out = t.cat((title_out,content_out),dim=1)
        logits = self.fc(conv_out)
        return F.log_softmax(logits)


In [29]:
from torch import optim

In [34]:
cnnt = CNNText()

optimizer = optim.Adam(cnnt.parameters(), lr=.001)
Loss = nn.NLLLoss()

for epoch in range(50):
    loss = 0
    
    t = ''.join(title[epoch])
    c = ''.join(content[epoch])
    T, C = variables_from_pair(t, c)
    print(T.unsqueeze(2))
    T = T.unsqueeze(2)
    C = C.unsqueeze(2)
    optimizer.zero_grad()
    
    out = cnnt(T, C)
    target = cla[epoch]
    loss += Loss(out, target)
    
    loss.backward()
    optimizer.step()
    
print("Loss is {} at {} epoch".format(loss, epoch))

Variable containing:
(0 ,.,.) = 
  2

(1 ,.,.) = 
  3

(2 ,.,.) = 
  4
[torch.LongTensor of size 3x1x1]



AssertionError: 